In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [4]:
!playwright install 

In [6]:
SEASONS=list(range(2014,2024))
SEASONS

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [7]:
DATA_DIR= 'data2'
STANDINGS_DIR= os.path.join(DATA_DIR, 'standings')
SCORES_DIR= os.path.join(DATA_DIR, 'score')

In [8]:
async def get_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        print(f"Fetching {url}, Attempt {i}")
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url, timeout=60000)  # Increase the timeout to 60,000 ms (60 seconds)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:  # Correct the exception name
            print(f"Timeout error on {url}")
        else:
            break
    return html

In [9]:
async def scrape_season(season):
    url= f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html= await get_html(url, '#content .filter')
    
    soup=BeautifulSoup(html)
    links=soup.find_all('a')
    href=[l['href']for l in links]
    standings_pages= [f'https://www.basketball-reference.com{l}'for l in href]
    
    for url in standings_pages:
        save_path= os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
            
        html = await get_html(url,'#all_schedule')
        with open (save_path, 'w+') as f:
            f.write(html)

In [10]:
standing_files= os.listdir(STANDINGS_DIR)

In [11]:
standing_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2015_games-march.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2014_games-june.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2016_games-april.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2015_games-january.html',
 'NBA_2015_games-june.html',
 'NBA_2017_games-february.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2015_games-february.html',
 'NBA_2018_games-april.html',
 'NBA_2014_games-march.html',
 'NBA_2014_games-february.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2014_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NB

In [12]:
#chat gpt
async def scrape_game(standing_files):
    with open(standing_files, 'r') as f:
        html= f.read()

    soup=BeautifulSoup(html)
    links= soup.find_all('a')
    hrefs=[l.get('href')for l in links]
    box_scores= [l for l in hrefs if l and "boxscore" in l and '.html' in l]
    box_scores= [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        if os.path.exists(save_path):
            continue

        html = await get_html(url, '#content')
        if not html:
            continue 

        with open(save_path, 'w+') as f:
            f.write(html)

In [14]:
for f in standing_files:
    filepath= os.path.join(STANDINGS_DIR, f)
    
    await scrape_game(filepath)

Fetching https://www.basketball-reference.com/boxscores/201503220MIL.html, Attempt 1


CancelledError: 